<a href="https://colab.research.google.com/github/Wangadeveloper/Failure-Classifier-to-Maintenance/blob/main/Untitled17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
sample=pd.read_csv('sample_submission.csv')

In [4]:
train.head(5)

,building_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,0,3,0,11,4,o,r,x,f,q,...,0,0,0,0,0,0,0,0,0,2
1,1,1,3,11,9,t,i,x,v,s,...,0,0,0,0,0,0,0,0,0,1
2,2,2,0,12,6,t,i,x,v,s,...,0,0,0,0,0,0,0,0,0,1
3,3,1,0,11,3,o,r,n,v,j,...,0,0,0,0,0,0,0,0,0,3
4,4,2,4,25,10,t,i,n,f,x,...,1,0,0,0,0,0,0,0,0,1


In [5]:
train.shape

(4000, 37)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 37 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   building_id                             4000 non-null   int64 
 1   count_floors_pre_eq                     4000 non-null   int64 
 2   age                                     4000 non-null   int64 
 3   area_percentage                         4000 non-null   int64 
 4   height_percentage                       4000 non-null   int64 
 5   land_surface_condition                  4000 non-null   object
 6   foundation_type                         4000 non-null   object
 7   roof_type                               4000 non-null   object
 8   ground_floor_type                       4000 non-null   object
 9   other_floor_type                        4000 non-null   object
 10  position                                4000 non-null   object
 11  plan

In [7]:
test.head(5)

,building_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,0,1,3,8,3,t,r,q,f,j,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,10,3,t,i,x,x,j,...,0,0,0,0,0,0,0,0,0,0
2,2,2,16,8,10,t,i,x,v,s,...,0,0,0,0,0,0,0,0,0,0
3,3,3,37,5,5,o,r,n,f,q,...,0,0,0,0,0,0,0,0,0,0
4,4,1,5,5,3,t,u,q,f,j,...,0,1,0,0,0,0,0,0,0,0


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 36 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   building_id                             1000 non-null   int64 
 1   count_floors_pre_eq                     1000 non-null   int64 
 2   age                                     1000 non-null   int64 
 3   area_percentage                         1000 non-null   int64 
 4   height_percentage                       1000 non-null   int64 
 5   land_surface_condition                  1000 non-null   object
 6   foundation_type                         1000 non-null   object
 7   roof_type                               1000 non-null   object
 8   ground_floor_type                       1000 non-null   object
 9   other_floor_type                        1000 non-null   object
 10  position                                1000 non-null   object
 11  plan_

In [10]:
X=train.drop('damage_grade',axis=1)
y=train['damage_grade']

In [11]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=.2,stratify=y)

In [12]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=5,
    learning_rate=0.1,
    #loss_function='CrossEntropy'
    )


In [13]:
cat_features = list(range(0, X.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]


In [14]:
clf.fit(x_train, y_train,
        cat_features=cat_features,
        eval_set=(x_test, y_test),
        verbose=False
)

In [15]:
print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

CatBoost model is fitted: True
CatBoost model parameters:
{'iterations': 5, 'learning_rate': 0.1}


In [16]:
from catboost import Pool
# Print the evaluation metrics
accuracy = metrics['Accuracy'][-1]

print(f'Accuracy: {accuracy:.2f}')

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=5]="t": Cannot convert 'b't'' to float